# <font color=#cd0000> Propósito principal </font>
- La idea de este librillo es preparar el entorno para realizar pruebas para cualquier DataSet a ser clasificado por cualquier codificación aplicada a SMTS

## <font color=#cd0000> Leyenda </font>
- Los apartados titulados con el código de colores HEX: `#cd0000` serán apartados que tendrán todos los librillos, en concreto, aquellos especificados en el apartado `Síntesis de los criterios usados` del trabajo.
- Los apartados titulados con el código de colores HEX: `#2451ff` serán apartados de conclusiones propias de este librillo resultado de aplicar un estudio personalizado para cada planteamiento.

# <font color=#cd0000> Prerrequisitos </font>
## <font color=#cd0000> Entorno de ejecución </font>
- Cambiamos el directorio raíz del librillo para acceder cómodamente a las funciones de utilidad.

In [1]:
import os

os.chdir('../..')

## <font color=#cd0000> Constantes y variables predefinidas </font>
- TODO -> Añadir SEED a todas las particiones.

In [2]:
HEARTBEAT_PATH = "data/heartbeat"
EPILEPSY_PATH = "data/epilepsy"
SEGUIMIENTO_OCULAR_PATH = "data/seguimiento-ocular/Data/Hospital"
SEGUIMIENTO_OCULAR_FOLDERS_ID = range(1, 12+1)

SEED = 1

# <font color=#cd0000> Carga del Dataset </font>
- TODO: Breve descripción

In [3]:
# TODO - Change with known data
from utils.load_data import import_epilepsy_dataset

train, test = import_epilepsy_dataset(EPILEPSY_PATH)


# <font color=#cd0000> Preprocesamiento </font>

## <font color=#cd0000> Eliminación de datos inválidos y valores atípicos </font>
- TODO: Breve descripción de qué es un dato inválido (-1's en columna, etc.)
- Eliminaremos aquellos valores fuera de los percentiles 5 y 95.
- TODO: Definiremos cuál será el límite de outliers permitido por serie temporal

In [4]:
# TODO - Remove invalid data

In [5]:
train.remove_outliers(headers=train.get_derived_data_columns()['attrs'], outliers_limit=0.3)
# test.remove_outliers(headers=test.get_derived_data_columns()['attrs']) -> Estos no los podemos alterar

In [6]:
import pandas as pd

# Remaining series
print("Previous number of series: {}".format(
    len(pd.unique(train.original_data['id']))))
print("Actual number of series: {}".format(
    len(pd.unique(train.derived_data['id']))))


Previous number of series: 137
Actual number of series: 78


## <font color=#cd0000> Resoluciones a aplicar </font>
- TODO:
  - Si las series son rápidas (muchos cambios en poco tiempo) especificar resoluciones altas (sin modificaciones).
  - Si las series son lentas (pocos cambios en mucho tiempo) especificar resoluciones bajas (eliminamos datos).

In [7]:
# Series lentas
train.reduce_sampling_rate(remove_one_each_n_samples=2)

In [8]:
train.derived_data

,id,TimeStamp,dim_0,dim_1,dim_2,class
0,0,1,0.40,0.01,0.51,epilepsy
1,0,3,0.47,-0.33,0.44,epilepsy
2,0,5,0.47,0.19,0.56,epilepsy
3,0,7,0.42,-0.58,0.63,epilepsy
4,0,9,0.37,-0.87,0.81,epilepsy
...,...,...,...,...,...,...
8029,136,197,-0.23,-0.75,-0.07,sawing
8030,136,199,1.22,-1.33,-0.26,sawing
8031,136,201,-0.08,-0.95,-0.55,sawing
8032,136,203,0.73,-0.18,-0.19,sawing


# <font color=#cd0000> Codificación </font>
- TODO: Breve descripción de la codificación

# <font color=#cd0000> División en ventanas </font>
- Solo aplicaremos enventanado si no ha sido aplicado anteriormente
- TODO: Especificar tamaño de ventana esperado como mejor y adjuntar otro tamaño de ventana para comparar (al menos 2 más)
- TODO: No es necesario aplicar siempre el enventanado, revisar análisis en profundidad.

In [ ]:
# Estudiamos eventos globales (series lentas)
ws_x = train.get_shortest_serie().shape[0]
train_ws_x, train_windows_per_serie_x = \
    train.split_into_windows(train.derived_data, window_size=ws_x)
test_ws_x, test_windows_per_serie_x =\
    test.split_into_windows(test.derived_data, window_size=ws_x)

# Estudiamos eventos locales (series rápidas)
ws_y = int(train.get_shortest_serie().shape[0]/2)
train_ws_y, train_windows_per_serie_y =\
    train.split_into_windows(train.derived_data, window_size=ws_y)
test_ws_y, test_windows_per_serie_y =\
    test.split_into_windows(test.derived_data, window_size=ws_y)

In [ ]:
from utils.data_extraction import Data

train_large_windows = Data(train_ws_x, train_windows_per_serie_x)
test_large_windows = Data(test_ws_x, test_windows_per_serie_x)

train_short_windows = Data(train_ws_y, train_windows_per_serie_y)
test_short_windows = Data(test_ws_y, test_windows_per_serie_y)

## <font color=#cd0000> Codificación sobre las ventanas </font>

In [11]:
from utils.codifications import temporal_trend_fn

train_large_windows.apply_codifications([temporal_trend_fn])
test_large_windows.apply_codifications([temporal_trend_fn])

# <font color=#cd0000> Diseño de la topología del bosque </font>
- Número de árboles inicial recomendado
- Número de símbolos inicial recomendado
- Número de árboles del segundo RandomForest recomendado

In [ ]:
# TODO

## <font color=#cd0000> Visualización de resultados preliminares </font>
- Visualización de sobreajuste
  - Para mitigarlo hay que disminuir el número de árboles del segundo RF (es el que clasifica)

In [ ]:
# TODO

# <font color=#cd0000> Randomized Search </font>
- Búsqueda de hiper-parámetros aleatoria con SMTS maximizando ``macro avg f1-score``

In [12]:

import random
import pickle
from smts_implementation.smts import SMTS
from utils.classifier_utils import (windowed_cross_val,
                                    compute_classification_reports_means)
from utils.plot_utils import pretty_print_classification_report_dict

PKL_DIR = 'pkl/SMTS/'


def smts_randomized_search_cv(
        windowed_series,
        relation_with_series,
        prefix,
        cv=5):
    global PKL_DIR
    all_clf_used = {}

    n_samples = 5
    j_ins_list = random.sample([20, 50, 100, 150, 200], n_samples)
    n_symbols_list = random.sample([20, 50, 100, 150, 200], n_samples)

    best_hyp_params = None
    best_score = 0
    for j_ins in j_ins_list:
        for n_symbols in n_symbols_list:
            clf_used = {}
            clf = SMTS(
                j_ins=j_ins,
                n_symbols=n_symbols,
                random_state=SEED
            )

            reports = windowed_cross_val(
                clf,
                windowed_series,
                relation_with_series,
                cv=cv,
                drop_columns=['class'],
                custom_estimator=True,
                seed=SEED
            )
            mean_report = compute_classification_reports_means(reports)
            all_clf_used[(j_ins, n_symbols)] = (clf_used, str(mean_report))

            if mean_report['macro avg']['f1-score'] >= best_score:
                best_score = mean_report['macro avg']['f1-score']
                best_hyp_params = (j_ins, n_symbols)
                best_report = mean_report

            print("\t\t--------------ACTUAL BEST: J_ins={}; N_symbols={}--------------"
                  .format(best_hyp_params[0], best_hyp_params[1]))
            pretty_print_classification_report_dict(best_report)
            print("\t\t--------------ITERATION: J_ins={}; N_symbols={}--------------"
                  .format(j_ins, n_symbols))
            pretty_print_classification_report_dict(mean_report)

    with open(PKL_DIR + prefix, 'wb') as file:
        pickle.dump(all_clf_used, file)

    return best_hyp_params, best_report


In [13]:
smts_randomized_search_cv(
    train_large_windows.derived_data,
    train_large_windows.derived_data_windows_per_serie,
    "smts_sample",
    cv=5)

		--------------ACTUAL BEST: J_ins=150; N_symbols=200--------------
Accuracy: 0.36666666666666675
epilepsy:
	precision: 0.24571428571428572
	recall: 0.24571428571428572
	f1-score: 0.24571428571428572
	support: 5.0
sawing:
	precision: 0.0
	recall: 0.0
	f1-score: 0.0
	support: 3.4
walking:
	precision: 0.5378571428571429
	recall: 0.5511904761904762
	f1-score: 0.5441729323308271
	support: 7.2
macro avg:
	precision: 0.26119047619047614
	recall: 0.2656349206349206
	f1-score: 0.2632957393483709
	support: 15.6
weighted avg:
	precision: 0.35866666666666674
	recall: 0.36666666666666675
	f1-score: 0.36245614035087725
	support: 15.6
		--------------ITERATION: J_ins=150; N_symbols=200--------------
Accuracy: 0.36666666666666675
epilepsy:
	precision: 0.24571428571428572
	recall: 0.24571428571428572
	f1-score: 0.24571428571428572
	support: 5.0
sawing:
	precision: 0.0
	recall: 0.0
	f1-score: 0.0
	support: 3.4
walking:
	precision: 0.5378571428571429
	recall: 0.5511904761904762
	f1-score: 0.544172932330

# <font color=#cd0000> Análisis de resultados </font>
- TODO - Un breve análisis de los resultados obtenidos para las diferentes resoluciones, ventanas, ...
- Visualización de gráficos para determinar si se pueden obtener mejores resultados con una serie de hiper-parámetros concretos.

In [19]:
# TODO

# <font color=#cd0000> Conclusiones </font>
- TODO - Unas breves conclusiones sobre los resultados obtenidos (influencia de la codificación, ...)